## Get Probs
### Alexandra Higley

A function that uses isQSO_randomforest from cuts.py (desi targets tool) which can call the probabilities associated with QSO targets and return them as an array. It takes in a single parameter, filename, which is the path and name of the FITS file with the objects that you would like to find probabilities on.

In [1]:
import fitsio
import numpy.lib.recfunctions as rfn
import numpy as np
from cuts import _prepare_optical_wise
from cuts import isQSO_randomforest
from astropy.table import Table, Column
import glob
import os

ModuleNotFoundError: No module named 'fitsio'

First, the desired FITS file is read in as **a**. Then, for prepare-optical-wise to work, we need to rename the headers BRICK_OBJID and MORPHTYPE to OBJID and TYPE. Once that is done, we can call the funciton ```_prepare_optical_wise``` from ```cuts.py``` to prepare the object measurements for ```isQSO_randomforest```. It will return all the necessary measurements, which can then be used as parameters for ```isQSO_randomforest```. For example, gflux is set equal to gflux, the latter of which was an output from ```_prepare_optical_wise```. Additionally, ```return_probs``` is set to **True** so that it will then return an array of probabilities— one for each object in the FITS file. It will also tell you how many of the probabilities are non-zero.

The second part of the function creates a new FITS table which contains the DESI_TARGET, the TARGETID, RA, DEC and a new header I call PROB, which are the probabilities returned from ```isQSO_randomforest```. This new file will be called ```targetprobs.fits```

In [1]:
def probtable(filename):

    a = fitsio.read(filename)

    #Have to rename certain colnames to make it compatible with prepare_optical and isQSO_randomforest
    objects = rfn.rename_fields(a,{'BRICK_OBJID':'OBJID', 'MORPHTYPE':'TYPE'})

    #using prepare_optical to prep targets
    photsys_north, photsys_south, obs_rflux, gflux, rflux, zflux, w1flux, w2flux, gfiberflux, rfiberflux, zfiberflux, gfibertotflux, rfibertotflux, zfibertotflux, objtype, release, ra, dec, gfluxivar, rfluxivar, zfluxivar, w1fluxivar, gnobs, rnobs, znobs, gfracflux, rfracflux, zfracflux, gfracmasked, rfracmasked, zfracmasked, gfracin, rfracin, zfracin, gallmask, rallmask, zallmask,gsnr, rsnr, zsnr, w1snr, w2snr, dchisq, deltaChi2, maskbits, refcat = _prepare_optical_wise(objects, mask=True)
    
    #calling probabilities 
    south, probs = isQSO_randomforest(gflux=gflux, rflux=rflux, zflux=zflux,maskbits=maskbits, w1flux=w1flux, w2flux=w2flux, objtype=objtype, gnobs=gnobs, rnobs=rnobs, znobs=znobs,primary=None, ra=ra, dec=dec, south= True, return_probs= True)

    print("number of nonzero probs are: " + str(np.sum(probs > 0)))
    
    #making fits file with found probabilities 
    data = Table()
    data['DESI_TARGET'] = a['DESI_TARGET']
    data['TARGETID'] = a['TARGETID']
    data['RA'] = a['RA']
    data['DEC'] = a['DEC']
    data['PROB'] = probs

    data.write('targetprobs.fits', format='fits')